# install

In [7]:
!pip install torch
!pip install transformers
!pip install sentencepiece
!pip install protobuf
!pip install gensim
!pip install -U scikit-learn
!pip install pandas

     ---------------------------------------- 0.0/123.5 kB ? eta -:--:--
     -------------------------------------- 123.5/123.5 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ----- ---------------------------------- 1.1/7.9 MB 34.0 MB/s eta 0:00:01
   ------------ --------------------------- 2.5/7.9 MB 32.3 MB/s eta 0:00:01
   --------------------- ------------------ 4.3/7.9 MB 39.3 MB/s eta 0:00:01
   ---------------------------- ----------- 5.7/7.9 MB 40.8 MB/s eta 0:00:01
   ----------------------------------- ---- 6.9/7.9 MB 40.5 MB/s eta 0:00:01
   ---------------------------------------  7.9/7.9 MB 38.9 MB/s eta 0:00:01
   ----------------

# data_reader

In [10]:
import itertools
import dataclasses
from typing import Iterable, Optional, Union, Dict, Tuple, List


@dataclasses.dataclass
class Sentence:
    id: int
    text: str
    context: str
    label: str

    def to_dict(self) -> Dict[str, Union[str, int]]:
        return dataclasses.asdict(self)


def balance_targets(sentences: Iterable[Sentence], method: str = "downsample_o_cat", shuffle=True) \
        -> Iterable[Sentence]:
    """
    Oversamples and/or undersamples training sentences by a number of targets.
    This is useful for linear shallow classifiers, that are prone to simply overfit the most-occurring category.
    See the source code for a documentation of resample methods logic
    :param shuffle: whether to shuffle the output
    :param sentences: sentences to resample
    :param method: resample method, one of {downsample_o_cat, downsample_o_pzk_cats, all_upsampled, remove_o_cat}
    :return: resampled, possibly shuffled input sentences
    """
    import random
    # take the second-top count from categories apart from "Other"
    targets = [s.label for s in sentences]
    second_top_count = sorted([sum([target == cat for target in targets]) for cat in set(targets) - {"O"}])[-2]
    if method == "downsample_o_cat":
        # downsample "other" category to second-most-occurring category count
        out_sentences = list((random.sample([s for s in sentences if s.label == "O"], second_top_count) +
                         [s for s in sentences if s.label != "O"]))
    elif method == "downsample_o_pzk_cats":
        # downsample "other" + "P_ZK" (experience description) category to third-most-occurring category count
        out_sentences = list((random.sample([s for s in sentences if s.label == "O"], second_top_count) +
                         [s for s in sentences if s.label != "O"]))
        out_sentences = list((random.sample([s for s in out_sentences if s.label == "P_ZK"], second_top_count) +
                         [s for s in out_sentences if s.label != "P_ZK"]))
    elif method == "all_upsampled":
        # upsample all categories to a count of most-occurring one (presumably "other" category)
        from itertools import chain
        out_sentences = list(itertools.chain(*[random.choices([s for s in sentences if s.label == cat],
                                                              k=second_top_count) for cat in set(targets)]))
    elif method == "remove_o_cat":
        # completely remove sentences of "other" category
        out_sentences = [s for s in sentences if s.label != "O"]
    else:
        out_sentences = sentences
    if shuffle:
        # random shuffle output sentences
        random.shuffle(out_sentences)
    return out_sentences


def get_sentence_vertical(sentences_dir: str, confidence_thrd: Optional[int] = 0) -> 'DataFrame':
    """
    Creates a tab-separated csv table with sentences_text, tags, users and sources, in out_table_path
    :param sentences_dir: directory of input sentences, divided to [train, val, test] subdirectories
    :param confidence_thrd: minimal mean confidence threshold of the retrieved sentences
    :return: Dataframe with attributes of retrieved sentences
    """
    from itertools import chain
    import pandas as pd  # if you need this, run 'pip install pandas==1.2.1'
    from utils.dataset import ReflexiveDataset

    sentences_splits = [ReflexiveDataset.sentences_from_tsv(sentences_dir, dataset_type, confidence_thrd,
                                                            use_context=True)
                        for dataset_type in ["train", "val", "test"]]
    out_vertical = pd.DataFrame.from_records([s.to_dict() for s in chain(*sentences_splits)])
    return out_vertical


def split_text_to_sentence_context(text: str, sep_chars: Tuple[str] = (".", "?", "!")) -> List[Tuple[str, str]]:
    """
    Splits the input text to sentences with the corresponding context,
    in the format compliant with the training of NeuralClassifier
    :param text: Full input paragraph, e.g. whole reflective diary, to extract the sentences to classify
    :param sep_chars: characters separating potential sentences
    """
    out_sentences = []
    current_sent = []
    words = text.split()

    for w_i, word in enumerate(words):
        current_sent.append(word)
        is_last_or_is_upper = (w_i == len(words)-1 or words[w_i+1][0].isupper())
        if any(word.endswith(mark) for mark in sep_chars) and is_last_or_is_upper:
            out_sentences.append(" ".join(current_sent))
            current_sent = []

    for sent_i, sent in enumerate(out_sentences):
        context = " ".join(out_sentences[sent_i-2:sent_i+2])
        yield sent, context


# dataset

In [11]:
import os
from enum import Enum
from pathlib import Path
from typing import List, Union, Dict
import ast

from filelock import FileLock
from torch.utils.data import Dataset
from transformers import AutoTokenizer, PreTrainedTokenizer, InputFeatures, logging
import pandas as pd
# from .data_reader import Sentence

logger = logging.get_logger()

# original labels:
# LABELS = ["O", "OS_PRES", "PERS", "POC", "P_ZK", "REF_P", "UV_OBT", "VY_IN", "VY_VY"]

LABELS = ["Other", "Belief", "Perspective", "Feeling", "Experience",
          "Reflection", "Difficulty", "Intention", "Learning"]


class Split(Enum):
    train = "train"
    eval = "eval"
    test = "test"


class ReflexiveDataset(Dataset):
    
    def __init__(self, sentences_dir: str, dataset_type: str, cache_dir: str, label_list: List[str],
                 tokenizer: Union[AutoTokenizer, PreTrainedTokenizer],
                 use_context=True, mean_confidence_threshold: int = 5):
        self.sentences_dir = sentences_dir
        self.confidence_thrd = mean_confidence_threshold
        self.dataset_type = dataset_type
        self.tokenizer = tokenizer
        self.label_list = label_list
        self.label_map = {label: i for i, label in enumerate(label_list)}
        self.use_context = use_context

        if not Path(cache_dir).exists():
            Path(cache_dir).mkdir(parents=True, exist_ok=True)

        cached_features_file = os.path.join(
            cache_dir,
            "cached_{}_{}_{}".format(dataset_type, tokenizer.__class__.__name__, str(self.tokenizer.model_max_length)),
        )
        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):
            logger.info(f"Creating features from reflexive diaries")
            self.sentences = self.sentences_from_tsv(sentences_dir, dataset_type, self.confidence_thrd, self.use_context)
            self.features = self.convert_examples_to_features(self.sentences)

    @staticmethod
    def sentences_from_tsv(sentences_dir: str, dataset_type: str,
                           confidence_thrd: int, use_context: bool) -> List[Sentence]:
        """Creates sentences for the training, eval and test sets."""
        tsv_path = os.path.join(sentences_dir, dataset_type, "sentences.tsv")
        df = pd.read_csv(tsv_path, sep='\t')
        df.sentence = df.sentence.fillna("")
        df.context = df.context.fillna("")
        sentences = []
        # group by sources, iterate every group separately, to avoid context overlays
        for idx, row in enumerate(df.itertuples()):
            confidences = ast.literal_eval(row.confidence)
            if sum(confidences) / len(confidences) >= confidence_thrd:
                sentences.append(Sentence(id=row.idx, text=row.sentence,
                                          context=row.context if use_context else None,
                                          label=row.y))
        logger.info("Retrieving %s of all %s %s sentences, over threshold %s" %
                    (len(sentences), len(df), dataset_type, confidence_thrd))
        return sentences
    
    def convert_examples_to_features(self, examples: List[Sentence]) -> List[Dict[str, List[int]]]:

        batch_encoding = self.tokenizer(
            text=[example.text.strip() for example in examples],
            text_pair=[e.context.strip() for e in examples] if self.use_context else None,
            padding="max_length",
            truncation=True,
        )

        features = []
        for i in range(len(examples)):
            inputs = {k: batch_encoding[k][i] for k in batch_encoding if k != "token_type_ids"}
            inputs["label"] = self.label_map[examples[i].label]
            features.append(inputs)

        for i, example in enumerate(examples[:5]):
            logger.info("*** Example ***")
            logger.info("id: %s" % (example.id))
            logger.info("features: %s" % features[i])

        return features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, i) -> InputFeatures:
        return self.features[i]


# shallow_classifier

In [12]:
from typing import List, Optional
from gensim import corpora
from gensim import matutils

import numpy as np

#from .utils.dataset import ReflexiveDataset
#from .utils.data_reader import Sentence


class ShallowClassifier:
    word_dictionary = None

    def __init__(self, classifier, use_context: bool, bow_size: int,
                 sentences_dir: Optional[str] = None, lang: str = "cze"):
        self.classifier = classifier
        self.sentences_dir = sentences_dir
        self.use_context = use_context
        self.bow_size = bow_size
        self.lang = lang

    def _preprocess_string(self, text: str) -> List[str]:
        from gensim.parsing import preprocess_string
        if self.lang == "cze":
            from utils.cs_stemmer import cz_stem
            return [cz_stem(word) for word in preprocess_string(text)]
        else:
            return preprocess_string(text)

    def _initialize_bow_model(self, sents: List[Sentence]):
        text_preprocessed = [self._preprocess_string(str(s.text)) for s in sents]
        contexts_preprocessed = [self._preprocess_string(str(s.context)) for s in sents]

        self.word_dictionary = corpora.Dictionary(text_preprocessed + contexts_preprocessed)
        # keep most-occurring 10k words
        # we need to check this with Ullmann
        self.word_dictionary.filter_extremes(keep_n=self.bow_size)

    def _vectorize_sentences(self, sents: List[Sentence]):
        text_preprocessed = [self._preprocess_string(s.text) for s in sents]
        # sparse matrix contains just pairs of co-occurrences
        sparse_matrix = [self.word_dictionary.doc2bow(t) for t in text_preprocessed]
        # we want to get natural, dense vectors for each document, containing the most-frequent num_terms
        dense_matrix = matutils.corpus2dense(sparse_matrix, num_terms=self.bow_size).transpose()
        if not self.use_context:
            return dense_matrix
        else:
            # the same for contextual vectors
            text_preprocessed_c = [self._preprocess_string(str(s.context)) for s in sents]
            # sparse matrix contains just pairs of co-occurrences
            sparse_matrix_c = [self.word_dictionary.doc2bow(t) for t in text_preprocessed_c]
            # we want to get natural, dense vectors for each document, containing the most-frequent num_terms
            dense_matrix_c = matutils.corpus2dense(sparse_matrix_c, num_terms=self.bow_size).transpose()

            # concat textual and contextual vectors horizontally
            return np.hstack([dense_matrix, dense_matrix_c])

    def train(self, in_sentences: List[Sentence] = None, confidence_thrd: int = 5):
        if in_sentences is None:
            # get the dataset from outside
            sentences = ReflexiveDataset.sentences_from_tsv(self.sentences_dir, "train",
                                                            confidence_thrd, self.use_context)
        else:
            # user gets the dataset himself
            sentences = in_sentences

        self._initialize_bow_model(sentences)
        vectors = self._vectorize_sentences(sentences)
        self.classifier.fit(vectors, [s.label for s in in_sentences])

    def predict(self, sentences: List[Sentence]):
        vectors = self._vectorize_sentences(sentences)
        targets = self.classifier.predict(vectors)
        return targets




# train_eval_shallow_classifier

In [18]:
#from reflection_classification.utils.dataset import ReflexiveDataset
#from reflection_classification.shallow_classifier import ShallowClassifier

from sklearn.metrics import f1_score, classification_report
import argparse


if __name__ == "__main__":
    argparser = "--classifier=random_forrest \ --sentences_dir=reflection-classification/data/sentences/en \--train_confidence_threshold=5 \--test_confidence_threshold=5 \--use_context=True \--vocabulary_size=800"

    argparser.add_argument('--classifier', type=str,
                           help='Classifier to use. One of: {random_forrest, logistic_regression, '
                                'naive_bayes, support_vector_classifier}', required=True)
    argparser.add_argument('--sentences_dir', type=str, required=True,
                           help='Directory with {split}/sentence.tsv of annotated sentences')
    argparser.add_argument('--train_confidence_threshold', type=int,
                           help='Minimal confidence threshold for sentences to train on.',
                           default=5)
    argparser.add_argument('--test_confidence_threshold', type=int,
                           help='Minimal confidence threshold for sentences to test on.',
                           default=5)
    argparser.add_argument('--use_context', type=bool, help='Whether the model was trainer using context.',
                           default=True)
    argparser.add_argument('--language', type=str, help='Language to decide on how to remove stemming.',
                           default="en")
    argparser.add_argument('--vocabulary_size', type=int,
                           help='Number of top-n most-occurring words used '
                                'to create Bag of Words representation for classification',
                           default=300)
    args = argparser.parse_args()

    if args.classifier == 'random_forrest':
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier()
    elif args.classifier == 'logistic_regression':
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(max_iter=10e4)
    elif args.classifier == 'naive_bayes':
        from sklearn.naive_bayes import MultinomialNB
        classifier = MultinomialNB()
    elif args.classifier == 'support_vector_classifier':
        from sklearn.svm import SVC
        classifier = SVC()
    else:
        raise ValueError("Unrecognized classifier: %s" % args.classifier)

    train_sentences = ReflexiveDataset.sentences_from_tsv(args.sentences_dir, "train",
                                                          args.train_confidence_threshold, args.use_context)
    test_sentences = ReflexiveDataset.sentences_from_tsv(args.sentences_dir, "test",
                                                         args.test_confidence_threshold, args.use_context)

    cfr = ShallowClassifier(classifier=classifier, use_context=args.use_context, bow_size=args.vocabulary_size,
                            lang=args.language)
    cfr.train(train_sentences)
    pred_targets = cfr.predict(test_sentences)
    true_targets = [s.label for s in test_sentences]
    objective_val = f1_score(true_targets, pred_targets, average='micro')
    print("Evaluating on %s sentences" % len(test_sentences))
    print("Classification report: \n%s" % classification_report(true_targets, pred_targets))
    print(objective_val)


AttributeError: 'str' object has no attribute 'add_argument'

In [15]:
%tb

SystemExit: 2